In [2]:
pip install pandas scikit-learn xgboost openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 3.4 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

# Step 1: Load Data
train = pd.read_csv("Train.csv", low_memory=False)

#  Step 2: Initial Cleaning
train = train.loc[:, train.isnull().mean() < 0.9]  # Drop columns with >90% missing
train.drop(columns=['SalesID', 'MachineID', 'ModelID'], inplace=True, errors='ignore')  # Drop IDs

# Parse saledate
train['saledate'] = pd.to_datetime(train['saledate'], errors='coerce')
train['sale_year'] = train['saledate'].dt.year
train['sale_month'] = train['saledate'].dt.month
train['sale_dayofweek'] = train['saledate'].dt.dayofweek
train.drop(columns='saledate', inplace=True)

# Step 3: Handle NaNs
for col in train.select_dtypes(include='number').columns:
    train[col] = train[col].fillna(train[col].median())

for col in train.select_dtypes(include='object').columns:
    train[col] = train[col].astype(str).str.lower().str.strip()
    train[col] = train[col].replace({'#####': np.nan, 'nan': np.nan, 'none': np.nan, '': np.nan})
    train[col] = train[col].fillna(train[col].mode()[0])
    freq = train[col].value_counts()
    rare = freq[freq < 50].index
    train[col] = train[col].apply(lambda x: 'other' if x in rare else x)

# Step 4: Encode categoricals
train = pd.get_dummies(train, drop_first=True)

# Step 5: Handle outliers
def cap_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower, upper)

for col in train.select_dtypes(include='number').columns:
    if col != 'SalePrice':
        train[col] = cap_outliers(train[col])

#  Step 6: Train/Validation Split
X = train.drop(columns='SalePrice')
y = train['SalePrice']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def calculate_rmsle(y_true, y_pred):
    y_true = np.clip(y_true, 1, None)
    y_pred = np.clip(y_pred, 1, None)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))

In [10]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
preds_lr = linreg.predict(X_val)
rmsle_lr = calculate_rmsle(y_val, preds_lr)
print(f"📈 Linear Regression RMSLE: {rmsle_lr:.4f}")

📈 Linear Regression RMSLE: 0.8162


In [22]:
rf = RandomForestRegressor(
    n_estimators=100,
    max_features='sqrt',
    max_depth=64,
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train, y_train)
preds_rf = rf.predict(X_val)
rmsle_rf = calculate_rmsle(y_val, preds_rf)
print(f"Random Forest RMSLE: {rmsle_rf:.4f}")

 Random Forest RMSLE: 0.2581


In [23]:
xgb = XGBRegressor(
    n_estimators=100,
    tree_method='hist',
    max_depth=15,
    learning_rate=0.1,
    n_jobs=-1,
    verbosity=0,
    random_state=42
)
xgb.fit(X_train, y_train)
preds_xgb = xgb.predict(X_val)
rmsle_xgb = calculate_rmsle(y_val, preds_xgb)
print(f"XGBoost RMSLE: {rmsle_xgb:.4f}")


XGBoost RMSLE: 0.2218


In [4]:
final_model = XGBRegressor(n_estimators=30,max_depth=15,random_state=42, verbosity=0)
final_model.fit(X, y)

# Load test data
test = pd.read_csv("Test.csv")
sales_ids = test['SalesID']  # Save IDs

# Repeat cleaning for test
test.drop(columns=['SalesID', 'MachineID', 'ModelID'], inplace=True, errors='ignore')
test['saledate'] = pd.to_datetime(test['saledate'], errors='coerce')
test['sale_year'] = test['saledate'].dt.year
test['sale_month'] = test['saledate'].dt.month
test['sale_dayofweek'] = test['saledate'].dt.dayofweek
test.drop(columns='saledate', inplace=True)

for col in test.select_dtypes(include='number').columns:
    test[col] = test[col].fillna(test[col].median())

for col in test.select_dtypes(include='object').columns:
    test[col] = test[col].astype(str).str.lower().str.strip()
    test[col] = test[col].replace({'#####': np.nan, 'nan': np.nan, 'none': np.nan, '': np.nan})
    test[col] = test[col].fillna(test[col].mode()[0])
    freq = test[col].value_counts()
    rare = freq[freq < 50].index
    test[col] = test[col].apply(lambda x: 'other' if x in rare else x)

# Match columns
test = pd.get_dummies(test)
test = test.reindex(columns=X.columns, fill_value=0)

# Predict
predictions = final_model.predict(test)
predictions = np.maximum(predictions, 0)  # ensure no negative prices

# Save output
output_df = pd.DataFrame({
    'SalesID': sales_ids,
    'SalePrice': predictions
})
output_df.to_csv("test_predictions.csv", index=False)
print("✅ test_predictions.csv saved.")

✅ test_predictions.csv saved.


In [20]:
from google.colab import files
files.download("test_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>